<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 11


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

Создать базовый класс Customer в C#, который будет представлять информацию о
клиентах или покупателях. На основе этого класса разработать 2-3 производных
класса, демонстрирующих принципы наследования и полиморфизма. В каждом из
классов должны быть реализованы новые атрибуты и методы, а также
переопределены некоторые методы базового класса для демонстрации
полиморфизма.

Требования к базовому классу Customer:

• Атрибуты: Идентификатор клиента (CustomerId), Имя (Name), Электронная
почта (Email).

• Методы:
o
o GetFullName(): метод для получения полного имени клиента.
o UpdateEmail(string newEmail): метод для обновления электронной
почты клиента.
o ViewProfile(): метод для просмотра профиля клиента.
Требования к производным классам:
1. VIPКлиент (VipCustomer): Должен содержать дополнительные атрибуты,
такие как Баланс лояльности (LoyaltyPoints). Метод ViewProfile() должен быть
переопределен для отображения дополнительной информации о VIPклиенте.
2. ОбычныйКлиент (RegularCustomer): Должен содержать дополнительные
атрибуты, такие как Дата регистрации (RegistrationDate).
Метод UpdateEmail() должен быть переопределен для добавления
информации о дате последнего обновления электронной почты.
3. ГрупповойКлиент (GroupCustomer) (если требуется третий класс): Должен
содержать дополнительные атрибуты, такие как Название группы
(GroupName). Метод GetFullName() должен быть переопределен для
отображения названия группы вместо имени клиента.

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) создайте явную реализации интерфейса и управление зависимостями 


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [4]:
using System;
using System.Collections.Generic;

// ИНТЕРФЕЙСЫ для явной реализации
public interface ICustomerNotificationService
{
    void SendWelcomeMessage(int customerId);
    void SendPromotion(int customerId, string promotion);
}

public interface ICustomerAuditService
{
    void LogCustomerAction(int customerId, string actionType, string details, DateTime timestamp);
}

public interface ICustomerAnalytics
{
    decimal CalculateCustomerValue(int customerId);
    int GetCustomerActivityScore(int customerId);
}

public interface IDiscountEligible
{
    decimal GetDiscountPercent();
    bool CanApplyDiscount(decimal purchaseAmount);
}

public interface ILoyaltyEarning
{
    int LoyaltyPoints { get; }
    void AddLoyaltyPoints(int points, string reason);
    bool RedeemLoyaltyPoints(int points);
}

// СЕРВИСЫ с внедрением зависимостей
public class EmailCustomerNotificationService : ICustomerNotificationService
{
    public void SendWelcomeMessage(int customerId)
    {
        Console.WriteLine($"Приветственное письмо отправлено клиенту ID: {customerId}");
    }

    public void SendPromotion(int customerId, string promotion)
    {
        Console.WriteLine($"Акция для клиента ID {customerId}: {promotion}");
    }
}

public class DatabaseAuditService : ICustomerAuditService
{
    public void LogCustomerAction(int customerId, string actionType, string details, DateTime timestamp)
    {
        Console.WriteLine($"Аудит: Клиент {customerId}, Действие: {actionType}, Детали: {details}, Время: {timestamp:HH:mm:ss}");
    }
}

public class CustomerAnalyticsService : ICustomerAnalytics
{
    public decimal CalculateCustomerValue(int customerId)
    {
        return (customerId % 10) * 1000m;
    }

    public int GetCustomerActivityScore(int customerId)
    {
        return 100 - (customerId % 20) * 5;
    }
}

// БАЗОВЫЙ КЛАСС Customer
public class Customer : IDiscountEligible, ILoyaltyEarning
{
    // БАЗОВЫЕ АТРИБУТЫ из задания
    public int CustomerId { get; protected set; }
    public string Name { get; protected set; }
    public string Email { get; protected set; }
    
    // ДОПОЛНИТЕЛЬНЫЕ АТРИБУТЫ
    public string Phone { get; protected set; }
    public DateTime RegistrationDate { get; protected set; }
    public bool IsActive { get; protected set; }
    public string Address { get; protected set; }
    public CustomerType Type { get; protected set; }
    public string SocialMedia { get; protected set; }
    public List<string> Preferences { get; private set; }
    public int LoginCount { get; private set; }
    public DateTime LastLoginDate { get; private set; }
    public decimal TotalPurchasesAmount { get; protected set; }
    public int TotalPurchasesCount { get; protected set; }
    
    // Поле для хранения баллов лояльности
    private int _loyaltyPoints;
    
    // ЗАВИСИМОСТИ
    protected readonly ICustomerNotificationService _notificationService;
    protected readonly ICustomerAuditService _auditService;
    
    // Enum для типа клиента
    public enum CustomerType
    {
        Individual,
        Business,
        Government
    }
    
    // КОНСТРУКТОР с Dependency Injection
    public Customer(int customerId, string name, string email, CustomerType type,
                   ICustomerNotificationService notificationService, 
                   ICustomerAuditService auditService)
    {
        CustomerId = customerId;
        Name = name;
        Email = email;
        Type = type;
        Phone = "";
        Address = "";
        SocialMedia = "";
        RegistrationDate = DateTime.Now;
        IsActive = true;
        Preferences = new List<string>();
        LoginCount = 0;
        LastLoginDate = DateTime.MinValue;
        TotalPurchasesAmount = 0;
        TotalPurchasesCount = 0;
        _loyaltyPoints = 0;
        
        _notificationService = notificationService;
        _auditService = auditService;
        
        _auditService.LogCustomerAction(customerId, "CUSTOMER_CREATED", 
            $"Создан клиент: {name} ({email})", DateTime.Now);
        _notificationService.SendWelcomeMessage(customerId);
    }
    
    // БАЗОВЫЕ МЕТОДЫ из задания
    public virtual string GetFullName()
    {
        return Name;
    }
    
    public virtual void UpdateEmail(string newEmail)
    {
        if (ValidateEmail(newEmail))
        {
            string oldEmail = Email;
            Email = newEmail;
            _auditService.LogCustomerAction(CustomerId, "EMAIL_UPDATED", 
                $"Email изменен: {oldEmail} -> {newEmail}", DateTime.Now);
            Console.WriteLine($"Email обновлен: {oldEmail} -> {newEmail}");
        }
        else
        {
            Console.WriteLine("Некорректный email адрес");
        }
    }
    
    public virtual void ViewProfile()
    {
        Console.WriteLine("\n======================================");
        Console.WriteLine($"ПРОФИЛЬ КЛИЕНТА ID: {CustomerId}");
        Console.WriteLine("======================================");
        Console.WriteLine($"Имя: {GetFullName()}");
        Console.WriteLine($"Email: {Email}");
        Console.WriteLine($"Телефон: {Phone}");
        Console.WriteLine($"Адрес: {Address}");
        Console.WriteLine($"Тип: {Type}");
        Console.WriteLine($"Соцсети: {SocialMedia}");
        Console.WriteLine($"Дата регистрации: {RegistrationDate:dd.MM.yyyy}");
        Console.WriteLine($"Активен: {(IsActive ? "Да" : "Нет")}");
        Console.WriteLine($"Предпочтения: {Preferences.Count} шт");
        Console.WriteLine($"Логинов: {LoginCount}");
        Console.WriteLine($"Последний вход: {(LastLoginDate == DateTime.MinValue ? "Никогда" : LastLoginDate.ToString("dd.MM.yyyy HH:mm"))}");
        Console.WriteLine($"Покупок: {TotalPurchasesCount} на сумму {TotalPurchasesAmount:C}");
        Console.WriteLine($"Баллы лояльности: {_loyaltyPoints}");
        Console.WriteLine("======================================");
    }
    
    // ДОПОЛНИТЕЛЬНЫЕ МЕТОДЫ
    public virtual void UpdatePhone(string newPhone)
    {
        Phone = newPhone;
        _auditService.LogCustomerAction(CustomerId, "PHONE_UPDATED", 
            $"Телефон обновлен: {newPhone}", DateTime.Now);
        Console.WriteLine($"Телефон обновлен: {newPhone}");
    }
    
    public virtual void UpdateAddress(string newAddress)
    {
        Address = newAddress;
        _auditService.LogCustomerAction(CustomerId, "ADDRESS_UPDATED", 
            $"Адрес обновлен", DateTime.Now);
        Console.WriteLine($"Адрес обновлен: {newAddress}");
    }
    
    public virtual void RecordLogin()
    {
        LoginCount++;
        LastLoginDate = DateTime.Now;
        _auditService.LogCustomerAction(CustomerId, "LOGIN_RECORDED", 
            $"Клиент вошел в систему", DateTime.Now);
        Console.WriteLine($"{Name} вошел в систему. Всего входов: {LoginCount}");
    }
    
    public virtual void AddPurchase(decimal amount, string description = "Покупка")
    {
        if (amount > 0)
        {
            TotalPurchasesCount++;
            TotalPurchasesAmount += amount;
            _auditService.LogCustomerAction(CustomerId, "PURCHASE_ADDED", 
                $"Покупка: {description} на {amount:C}", DateTime.Now);
            
            int points = (int)(amount / 100);
            ((ILoyaltyEarning)this).AddLoyaltyPoints(points, $"Покупка: {description}");
            
            Console.WriteLine($"Добавлена покупка #{TotalPurchasesCount}: {description} на сумму {amount:C}");
        }
    }
    
    // ПЕРЕГРУЗКА методов
    public virtual void UpdateEmail(string newEmail, bool sendNotification)
    {
        string oldEmail = Email;
        UpdateEmail(newEmail);
        if (sendNotification)
        {
            Console.WriteLine($"Отправлено уведомление на старый email: {oldEmail}");
        }
    }
    
    public void ViewProfile(bool brief)
    {
        if (brief)
        {
            Console.WriteLine($"{GetFullName()} (ID: {CustomerId}, Email: {Email}, Тип: {Type})");
        }
        else
        {
            ViewProfile();
        }
    }
    
    // РЕАЛИЗАЦИЯ ИНТЕРФЕЙСА IDiscountEligible
    public virtual decimal GetDiscountPercent()
    {
        if (TotalPurchasesAmount > 100000) return 15;
        if (TotalPurchasesAmount > 50000) return 10;
        if (TotalPurchasesAmount > 10000) return 5;
        return 0;
    }
    
    public virtual bool CanApplyDiscount(decimal purchaseAmount)
    {
        return purchaseAmount >= 1000 && TotalPurchasesCount > 0;
    }
    
    // РЕАЛИЗАЦИЯ ИНТЕРФЕЙСА ILoyaltyEarning
    int ILoyaltyEarning.LoyaltyPoints => _loyaltyPoints;
    
    void ILoyaltyEarning.AddLoyaltyPoints(int points, string reason)
    {
        if (points > 0)
        {
            _loyaltyPoints += points;
            _auditService.LogCustomerAction(CustomerId, "LOYALTY_POINTS_ADDED", 
                $"Добавлено {points} баллов. Причина: {reason}", DateTime.Now);
            Console.WriteLine($"Добавлено {points} баллов лояльности. Всего: {_loyaltyPoints}");
        }
    }
    
    bool ILoyaltyEarning.RedeemLoyaltyPoints(int points)
    {
        if (points <= _loyaltyPoints && points > 0)
        {
            _loyaltyPoints -= points;
            _auditService.LogCustomerAction(CustomerId, "LOYALTY_POINTS_REDEEMED", 
                $"Списано {points} баллов", DateTime.Now);
            Console.WriteLine($"Списано {points} баллов. Осталось: {_loyaltyPoints}");
            return true;
        }
        Console.WriteLine("Недостаточно баллов лояльности");
        return false;
    }
    
    // Вспомогательные методы
    protected virtual bool ValidateEmail(string email)
    {
        return !string.IsNullOrEmpty(email) && email.Contains("@") && email.Contains(".");
    }
    
    public int GetCustomerAgeInDays()
    {
        return (DateTime.Now - RegistrationDate).Days;
    }
    
    public bool IsHighlyActive()
    {
        return LoginCount > 10 && GetCustomerAgeInDays() > 7;
    }
    
    public void Deactivate()
    {
        IsActive = false;
        _auditService.LogCustomerAction(CustomerId, "CUSTOMER_DEACTIVATED", 
            "Клиент деактивирован", DateTime.Now);
        _notificationService.SendPromotion(CustomerId, "Ваш аккаунт был деактивирован");
        Console.WriteLine($"Клиент {Name} деактивирован");
    }
    
    public void Activate()
    {
        IsActive = true;
        _auditService.LogCustomerAction(CustomerId, "CUSTOMER_ACTIVATED", 
            "Клиент активирован", DateTime.Now);
        _notificationService.SendPromotion(CustomerId, "Ваш аккаунт снова активен!");
        Console.WriteLine($"Клиент {Name} активирован");
    }
}

// 1. VIPКЛИЕНТ (VipCustomer)
public class VipCustomer : Customer
{
    // ДОПОЛНИТЕЛЬНЫЕ АТРИБУТЫ
    public string VipLevel { get; private set; }
    public decimal VipDiscount { get; private set; }
    public List<string> SpecialOffers { get; private set; }
    public DateTime LastSpecialOfferDate { get; private set; }
    public decimal MonthlySpendingLimit { get; private set; }
    public string PersonalManager { get; private set; }
    public List<string> PurchaseHistory { get; private set; }
    
    // КОНСТРУКТОР с Dependency Injection
    public VipCustomer(int customerId, string name, string email,
                      ICustomerNotificationService notificationService,
                      ICustomerAuditService auditService,
                      decimal monthlyLimit = 50000,
                      string personalManager = "Не назначен")
        : base(customerId, name, email, CustomerType.Individual, notificationService, auditService)
    {
        MonthlySpendingLimit = monthlyLimit;
        PersonalManager = personalManager;
        SpecialOffers = new List<string>();
        PurchaseHistory = new List<string>();
        LastSpecialOfferDate = DateTime.MinValue;
        UpdateVipLevel();
        
        _auditService.LogCustomerAction(customerId, "VIP_CUSTOMER_CREATED", 
            $"Создан VIP клиент с лимитом {monthlyLimit:C}", DateTime.Now);
    }
    
    // ПЕРЕОПРЕДЕЛЕНИЕ метода ViewProfile()
    public override void ViewProfile()
    {
        Console.WriteLine("\n======================================");
        Console.WriteLine($"ПРОФИЛЬ VIP КЛИЕНТА ID: {CustomerId}");
        Console.WriteLine("======================================");
        Console.WriteLine($"Имя: {GetFullName()}");
        Console.WriteLine($"Email: {Email}");
        Console.WriteLine($"Уровень VIP: {VipLevel}");
        Console.WriteLine($"Скидка VIP: {VipDiscount}%");
        Console.WriteLine($"Персональный менеджер: {PersonalManager}");
        Console.WriteLine($"Лимит трат в месяц: {MonthlySpendingLimit:C}");
        Console.WriteLine($"Спецпредложения: {SpecialOffers.Count} шт");
        Console.WriteLine($"История покупок: {PurchaseHistory.Count} записей");
        Console.WriteLine($"Дата регистрации: {RegistrationDate:dd.MM.yyyy}");
        Console.WriteLine($"Баллы лояльности: {((ILoyaltyEarning)this).LoyaltyPoints}");
        Console.WriteLine("======================================");
    }
    
    // НОВЫЕ МЕТОДЫ
    public void AddSpecialOffer(string offer)
    {
        SpecialOffers.Add(offer);
        LastSpecialOfferDate = DateTime.Now;
        _auditService.LogCustomerAction(CustomerId, "SPECIAL_OFFER_ADDED", 
            $"Добавлено спецпредложение: {offer}", DateTime.Now);
        Console.WriteLine($"Добавлено спецпредложение для VIP: {offer}");
    }
    
    public void SetPersonalManager(string managerName)
    {
        PersonalManager = managerName;
        _auditService.LogCustomerAction(CustomerId, "PERSONAL_MANAGER_SET", 
            $"Назначен персональный менеджер: {managerName}", DateTime.Now);
        _notificationService.SendPromotion(CustomerId, $"Вам назначен персональный менеджер: {managerName}");
        Console.WriteLine($"Назначен персональный менеджер: {managerName}");
    }
    
    public void UpdateMonthlyLimit(decimal newLimit)
    {
        MonthlySpendingLimit = newLimit;
        _auditService.LogCustomerAction(CustomerId, "MONTHLY_LIMIT_UPDATED", 
            $"Лимит обновлен: {newLimit:C}", DateTime.Now);
        Console.WriteLine($"Лимит трат обновлен: {newLimit:C}");
    }
    
    public override void AddPurchase(decimal amount, string description = "Покупка")
    {
        base.AddPurchase(amount, description);
        PurchaseHistory.Add($"{DateTime.Now:dd.MM.yyyy} - {description} - {amount:C}");
        
        int bonusPoints = (int)(amount / 50);
        ((ILoyaltyEarning)this).AddLoyaltyPoints(bonusPoints, "VIP бонус за покупку");
    }
    
    // Переопределение метода UpdatePhone
    public override void UpdatePhone(string newPhone)
    {
        base.UpdatePhone(newPhone);
        ((ILoyaltyEarning)this).AddLoyaltyPoints(100, "VIP бонус за обновление контактов");
        Console.WriteLine($"VIP бонус: 100 баллов за обновление телефона");
    }
    
    private void UpdateVipLevel()
    {
        if (TotalPurchasesAmount >= 100000)
        {
            VipLevel = "Platinum";
            VipDiscount = 25;
        }
        else if (TotalPurchasesAmount >= 50000)
        {
            VipLevel = "Gold";
            VipDiscount = 20;
        }
        else if (TotalPurchasesAmount >= 10000)
        {
            VipLevel = "Silver";
            VipDiscount = 15;
        }
        else
        {
            VipLevel = "Bronze";
            VipDiscount = 10;
        }
    }
    
    // Переопределение GetDiscountPercent для VIP
    public override decimal GetDiscountPercent()
    {
        decimal baseDiscount = base.GetDiscountPercent();
        return baseDiscount + VipDiscount;
    }
}

// 2. ОБЫЧНЫЙ КЛИЕНТ (RegularCustomer)
public class RegularCustomer : Customer
{
    // ДОПОЛНИТЕЛЬНЫЕ АТРИБУТЫ
    public DateTime? LastEmailUpdateDate { get; private set; }
    public decimal CreditLimit { get; private set; }
    public bool HasOverdraft { get; private set; }
    public List<string> Wishlist { get; private set; }
    public decimal MonthlyBudget { get; private set; }
    public decimal CurrentMonthSpending { get; private set; }
    
    // КОНСТРУКТОР с Dependency Injection
    public RegularCustomer(int customerId, string name, string email,
                          ICustomerNotificationService notificationService,
                          ICustomerAuditService auditService,
                          decimal creditLimit = 10000,
                          bool hasOverdraft = false,
                          decimal monthlyBudget = 5000)
        : base(customerId, name, email, CustomerType.Individual, notificationService, auditService)
    {
        LastEmailUpdateDate = null;
        CreditLimit = creditLimit;
        HasOverdraft = hasOverdraft;
        MonthlyBudget = monthlyBudget;
        CurrentMonthSpending = 0;
        Wishlist = new List<string>();
        
        _auditService.LogCustomerAction(customerId, "REGULAR_CUSTOMER_CREATED", 
            $"Создан обычный клиент с кредитным лимитом {creditLimit:C}", DateTime.Now);
    }
    
    // ПЕРЕОПРЕДЕЛЕНИЕ метода UpdateEmail()
    public override void UpdateEmail(string newEmail)
    {
        base.UpdateEmail(newEmail);
        LastEmailUpdateDate = DateTime.Now;
        Console.WriteLine($"Дата обновления email: {LastEmailUpdateDate:dd.MM.yyyy HH:mm:ss}");
        
        if (TotalPurchasesCount > 0)
        {
            AddPurchase(100, "Бонус за обновление email");
        }
    }
    
    // ПЕРЕОПРЕДЕЛЕНИЕ метода ViewProfile()
    public override void ViewProfile()
    {
        Console.WriteLine("\n======================================");
        Console.WriteLine($"ПРОФИЛЬ ОБЫЧНОГО КЛИЕНТА ID: {CustomerId}");
        Console.WriteLine("======================================");
        Console.WriteLine($"Имя: {GetFullName()}");
        Console.WriteLine($"Email: {Email}");
        Console.WriteLine($"Телефон: {Phone}");
        Console.WriteLine($"Кредитный лимит: {CreditLimit:C}");
        Console.WriteLine($"Овердрафт: {(HasOverdraft ? "Доступен" : "Не доступен")}");
        Console.WriteLine($"Месячный бюджет: {MonthlyBudget:C}");
        Console.WriteLine($"Траты в этом месяце: {CurrentMonthSpending:C}");
        Console.WriteLine($"Список желаний: {Wishlist.Count} товаров");
        Console.WriteLine($"Покупок всего: {TotalPurchasesCount} на сумму {TotalPurchasesAmount:C}");
        if (LastEmailUpdateDate.HasValue)
        {
            Console.WriteLine($"Email обновлялся: {LastEmailUpdateDate:dd.MM.yyyy}");
        }
        Console.WriteLine($"Баллы лояльности: {((ILoyaltyEarning)this).LoyaltyPoints}");
        Console.WriteLine("======================================");
    }
    
    // НОВЫЕ МЕТОДЫ
    public void AddToWishlist(string item)
    {
        Wishlist.Add(item);
        _auditService.LogCustomerAction(CustomerId, "ITEM_ADDED_TO_WISHLIST", 
            $"Добавлено в список желаний: {item}", DateTime.Now);
        Console.WriteLine($"Добавлено в список желаний: {item}");
    }
    
    public void UpdateCreditLimit(decimal newLimit)
    {
        CreditLimit = newLimit;
        _auditService.LogCustomerAction(CustomerId, "CREDIT_LIMIT_UPDATED", 
            $"Кредитный лимит обновлен: {newLimit:C}", DateTime.Now);
        _notificationService.SendPromotion(CustomerId, $"Ваш кредитный лимит увеличен до {newLimit:C}");
        Console.WriteLine($"Кредитный лимит обновлен: {newLimit:C}");
    }
    
    public void ToggleOverdraft(bool enable)
    {
        HasOverdraft = enable;
        string status = enable ? "включен" : "выключен";
        _auditService.LogCustomerAction(CustomerId, "OVERDRAFT_TOGGLED", 
            $"Овердрафт {status}", DateTime.Now);
        Console.WriteLine($"Овердрафт {status}");
    }
    
    public void SetMonthlyBudget(decimal budget)
    {
        MonthlyBudget = budget;
        CurrentMonthSpending = 0;
        _auditService.LogCustomerAction(CustomerId, "MONTHLY_BUDGET_SET", 
            $"Установлен месячный бюджет: {budget:C}", DateTime.Now);
        Console.WriteLine($"Установлен месячный бюджет: {budget:C}");
    }
    
    public override void AddPurchase(decimal amount, string description = "Покупка")
    {
        if (CurrentMonthSpending + amount > MonthlyBudget && MonthlyBudget > 0)
        {
            Console.WriteLine($"Превышение месячного бюджета! Бюджет: {MonthlyBudget:C}, Траты: {CurrentMonthSpending:C}");
            return;
        }
        
        base.AddPurchase(amount, description);
        CurrentMonthSpending += amount;
        
        if (TotalPurchasesCount % 5 == 0)
        {
            ((ILoyaltyEarning)this).AddLoyaltyPoints(50, "Бонус за каждую 5-ю покупку");
        }
    }
}

// 3. ГРУППОВОЙ КЛИЕНТ (GroupCustomer)
public class GroupCustomer : Customer
{
    // ДОПОЛНИТЕЛЬНЫЙ АТРИБУТ из задания
    public string GroupName { get; private set; }
    
    // ДОПОЛНИТЕЛЬНЫЕ АТРИБУТЫ
    public int MemberCount { get; private set; }
    public string ContactPerson { get; private set; }
    public List<string> Members { get; private set; }
    public decimal GroupDiscount { get; private set; }
    public string Industry { get; private set; }
    public decimal GroupBudget { get; private set; }
    public List<string> GroupProjects { get; private set; }
    public int ActiveProjects { get; private set; }
    
    // КОНСТРУКТОР с Dependency Injection
    public GroupCustomer(int customerId, string groupName, string email, string contactPerson,
                        int memberCount, string industry,
                        ICustomerNotificationService notificationService,
                        ICustomerAuditService auditService)
        : base(customerId, groupName, email, CustomerType.Business, notificationService, auditService)
    {
        GroupName = groupName;
        ContactPerson = contactPerson;
        MemberCount = memberCount;
        Industry = industry;
        GroupBudget = 0;
        Members = new List<string>();
        GroupProjects = new List<string>();
        ActiveProjects = 0;
        GroupDiscount = CalculateGroupDiscount();
        
        _auditService.LogCustomerAction(customerId, "GROUP_CUSTOMER_CREATED", 
            $"Создан групповой клиент: {groupName} ({memberCount} участников)", DateTime.Now);
    }
    
    // ПЕРЕОПРЕДЕЛЕНИЕ метода GetFullName()
    public override string GetFullName()
    {
        return $"Группа: {GroupName}";
    }
    
    // ПЕРЕОПРЕДЕЛЕНИЕ метода ViewProfile()
    public override void ViewProfile()
    {
        Console.WriteLine("\n======================================");
        Console.WriteLine($"ПРОФИЛЬ ГРУППОВОГО КЛИЕНТА ID: {CustomerId}");
        Console.WriteLine("======================================");
        Console.WriteLine($"Название группы: {GroupName}");
        Console.WriteLine($"Отрасль: {Industry}");
        Console.WriteLine($"Контактное лицо: {ContactPerson}");
        Console.WriteLine($"Email: {Email}");
        Console.WriteLine($"Телефон: {Phone}");
        Console.WriteLine($"Количество участников: {MemberCount}");
        Console.WriteLine($"Зарегистрировано: {Members.Count}");
        Console.WriteLine($"Бюджет группы: {GroupBudget:C}");
        Console.WriteLine($"Скидка группы: {GroupDiscount}%");
        Console.WriteLine($"Проектов: {GroupProjects.Count} (активных: {ActiveProjects})");
        Console.WriteLine($"Дата регистрации: {RegistrationDate:dd.MM.yyyy}");
        Console.WriteLine($"Активен: {(IsActive ? "Да" : "Нет")}");
        Console.WriteLine($"Общие покупки: {TotalPurchasesCount} на сумму {TotalPurchasesAmount:C}");
        Console.WriteLine("======================================");
    }
    
    // НОВЫЕ МЕТОДЫ
    public void AddMember(string memberName, string position = "Участник")
    {
        if (Members.Count < MemberCount)
        {
            Members.Add($"{memberName} - {position}");
            UpdateGroupDiscount();
            _auditService.LogCustomerAction(CustomerId, "MEMBER_ADDED", 
                $"Добавлен участник: {memberName} ({position})", DateTime.Now);
            Console.WriteLine($"Добавлен участник: {memberName} ({position})");
        }
        else
        {
            Console.WriteLine($"Достигнуто максимальное количество участников ({MemberCount})");
        }
    }
    
    public void AddProject(string projectName, decimal budget, bool isActive = true)
    {
        GroupProjects.Add($"{projectName} - {budget:C}");
        GroupBudget += budget;
        if (isActive) ActiveProjects++;
        
        _auditService.LogCustomerAction(CustomerId, "PROJECT_ADDED", 
            $"Добавлен проект: {projectName} с бюджетом {budget:C}", DateTime.Now);
        _notificationService.SendPromotion(CustomerId, $"Создан новый проект: {projectName}");
        
        Console.WriteLine($"Добавлен проект: {projectName} с бюджетом {budget:C}");
    }
    
    public void UpdateIndustry(string newIndustry)
    {
        Industry = newIndustry;
        _auditService.LogCustomerAction(CustomerId, "INDUSTRY_UPDATED", 
            $"Отрасль обновлена: {newIndustry}", DateTime.Now);
        Console.WriteLine($"Отрасль обновлена: {newIndustry}");
    }
    
    public void AllocateBudget(decimal amount)
    {
        GroupBudget += amount;
        _auditService.LogCustomerAction(CustomerId, "BUDGET_ALLOCATED", 
            $"Выделен бюджет: {amount:C}", DateTime.Now);
        Console.WriteLine($"Выделен бюджет: {amount:C}. Общий бюджет: {GroupBudget:C}");
    }
    
    public void SendGroupNotification(string message)
    {
        _auditService.LogCustomerAction(CustomerId, "GROUP_NOTIFICATION_SENT", 
            $"Отправлено уведомление группе: {message}", DateTime.Now);
        Console.WriteLine($"Уведомление для группы '{GroupName}': {message}");
        Console.WriteLine($"Отправлено {Members.Count} участникам");
    }
    
    private void UpdateGroupDiscount()
    {
        GroupDiscount = CalculateGroupDiscount();
    }
    
    private decimal CalculateGroupDiscount()
    {
        if (Members.Count >= 50) return 30;
        if (Members.Count >= 20) return 20;
        if (Members.Count >= 10) return 15;
        if (Members.Count >= 5) return 10;
        return 5;
    }
    
    // Переопределение GetDiscountPercent для групп
    public override decimal GetDiscountPercent()
    {
        decimal baseDiscount = base.GetDiscountPercent();
        return baseDiscount + GroupDiscount;
    }
}

// GENERIC МЕНЕДЖЕР КЛИЕНТОВ с Dependency Injection
public class CustomerManager<T> where T : Customer
{
    private List<T> _customers = new List<T>();
    private readonly ICustomerNotificationService _notificationService;
    private readonly ICustomerAuditService _auditService;
    private readonly ICustomerAnalytics _analyticsService;
    
    public CustomerManager(ICustomerNotificationService notificationService,
                          ICustomerAuditService auditService,
                          ICustomerAnalytics analyticsService)
    {
        _notificationService = notificationService;
        _auditService = auditService;
        _analyticsService = analyticsService;
    }
    
    public void AddCustomer(T customer)
    {
        _customers.Add(customer);
        _auditService.LogCustomerAction(customer.CustomerId, "CUSTOMER_ADDED_TO_MANAGER", 
            "Клиент добавлен в менеджер", DateTime.Now);
        Console.WriteLine($"Клиент добавлен в менеджер: {customer.GetFullName()}");
    }
    
    public T FindCustomerById(int customerId)
    {
        return _customers.Find(c => c.CustomerId == customerId);
    }
    
    public List<T> FindCustomersByName(string name)
    {
        return _customers.FindAll(c => 
            c.GetFullName().Contains(name, StringComparison.OrdinalIgnoreCase));
    }
    
    public void ShowAllCustomers()
    {
        Console.WriteLine($"\nВСЕ КЛИЕНТЫ В МЕНЕДЖЕРЕ ({typeof(T).Name}):");
        Console.WriteLine("======================================");
        foreach (var customer in _customers)
        {
            Console.WriteLine($"ID: {customer.CustomerId}, Имя: {customer.GetFullName()}, Email: {customer.Email}");
        }
        Console.WriteLine($"Всего: {_customers.Count} клиент(ов)");
        Console.WriteLine("======================================");
    }
    
    public void SendPromotionToAll(string promotion)
    {
        Console.WriteLine($"\nРассылка акции для всех клиентов:");
        Console.WriteLine("======================================");
        foreach (var customer in _customers)
        {
            _notificationService.SendPromotion(customer.CustomerId, promotion);
        }
        Console.WriteLine($"Акция отправлена {_customers.Count} клиентам");
        Console.WriteLine("======================================");
    }
    
    public void DisplayAnalytics()
    {
        Console.WriteLine($"\nАНАЛИТИКА КЛИЕНТОВ ({typeof(T).Name}):");
        Console.WriteLine("======================================");
        foreach (var customer in _customers)
        {
            decimal value = _analyticsService.CalculateCustomerValue(customer.CustomerId);
            int activity = _analyticsService.GetCustomerActivityScore(customer.CustomerId);
            Console.WriteLine($"{customer.GetFullName()}: Ценность = {value:C}, Активность = {activity}/100");
        }
        Console.WriteLine("======================================");
    }
    
    public int Count => _customers.Count;
}

// СЕРВИС ТРАНЗАКЦИЙ с Dependency Injection
public class CustomerTransactionService<T> where T : Customer
{
    private readonly ICustomerAuditService _auditService;
    private readonly ICustomerNotificationService _notificationService;
    
    public CustomerTransactionService(ICustomerAuditService auditService,
                                    ICustomerNotificationService notificationService)
    {
        _auditService = auditService;
        _notificationService = notificationService;
    }
    
    public bool ProcessPurchase(T customer, decimal amount, string item)
    {
        try
        {
            customer.AddPurchase(amount, item);
            _auditService.LogCustomerAction(customer.CustomerId, "PURCHASE_PROCESSED", 
                $"Покупка обработана: {item} за {amount:C}", DateTime.Now);
            return true;
        }
        catch (Exception ex)
        {
            Console.WriteLine($"Ошибка обработки покупки: {ex.Message}");
            return false;
        }
    }
    
    public void TransferPoints(T fromCustomer, T toCustomer, int points)
    {
        if (((ILoyaltyEarning)fromCustomer).RedeemLoyaltyPoints(points))
        {
            ((ILoyaltyEarning)toCustomer).AddLoyaltyPoints(points, $"Перевод от {fromCustomer.Name}");
            _auditService.LogCustomerAction(fromCustomer.CustomerId, "POINTS_TRANSFERRED", 
                $"Переведено {points} баллов клиенту {toCustomer.CustomerId}", DateTime.Now);
            _notificationService.SendPromotion(fromCustomer.CustomerId, $"Вы перевели {points} баллов");
            _notificationService.SendPromotion(toCustomer.CustomerId, $"Вам переведено {points} баллов");
            Console.WriteLine($"Успешно переведено {points} баллов");
        }
    }
}

// СИСТЕМА УПРАВЛЕНИЯ КЛИЕНТАМИ
public class CustomerManagementSystem
{
    private List<Customer> _allCustomers = new List<Customer>();
    private readonly ICustomerNotificationService _notificationService;
    private readonly ICustomerAuditService _auditService;
    private readonly ICustomerAnalytics _analyticsService;
    
    public CustomerManagementSystem(ICustomerNotificationService notificationService,
                                  ICustomerAuditService auditService,
                                  ICustomerAnalytics analyticsService)
    {
        _notificationService = notificationService;
        _auditService = auditService;
        _analyticsService = analyticsService;
    }
    
    public void AddCustomer(Customer customer)
    {
        _allCustomers.Add(customer);
        _auditService.LogCustomerAction(customer.CustomerId, "CUSTOMER_ADDED_TO_SYSTEM", 
            "Клиент добавлен в систему", DateTime.Now);
        Console.WriteLine($"Клиент добавлен в систему: {customer.GetFullName()}");
    }
    
    public void ShowSystemStatistics()
    {
        Console.WriteLine($"\nСТАТИСТИКА СИСТЕМЫ:");
        Console.WriteLine("======================================");
        Console.WriteLine($"Всего клиентов: {_allCustomers.Count}");
        
        var vipCount = 0;
        var regularCount = 0;
        var groupCount = 0;
        
        foreach (var customer in _allCustomers)
        {
            if (customer is VipCustomer) vipCount++;
            else if (customer is RegularCustomer) regularCount++;
            else if (customer is GroupCustomer) groupCount++;
        }
        
        Console.WriteLine($"VIP клиентов: {vipCount}");
        Console.WriteLine($"Обычных клиентов: {regularCount}");
        Console.WriteLine($"Групповых клиентов: {groupCount}");
        
        decimal totalValue = 0;
        foreach (var customer in _allCustomers)
        {
            totalValue += _analyticsService.CalculateCustomerValue(customer.CustomerId);
        }
        Console.WriteLine($"Общая ценность клиентов: {totalValue:C}");
        
        double totalActivity = 0;
        foreach (var customer in _allCustomers)
        {
            totalActivity += _analyticsService.GetCustomerActivityScore(customer.CustomerId);
        }
        double avgActivity = _allCustomers.Count > 0 ? totalActivity / _allCustomers.Count : 0;
        Console.WriteLine($"Средняя активность: {avgActivity:F1}/100");
        Console.WriteLine("======================================");
    }
    
    public void RunMonthlyReport()
    {
        Console.WriteLine($"\nМЕСЯЧНЫЙ ОТЧЕТ:");
        Console.WriteLine("======================================");
        foreach (var customer in _allCustomers)
        {
            Console.WriteLine($"{customer.GetFullName()}:");
            Console.WriteLine($"  Покупок за месяц: {customer.TotalPurchasesCount}");
            Console.WriteLine($"  Сумма покупок: {customer.TotalPurchasesAmount:C}");
            Console.WriteLine($"  Баллы лояльности: {((ILoyaltyEarning)customer).LoyaltyPoints}");
            
            if (customer.TotalPurchasesAmount > 10000)
            {
                _notificationService.SendPromotion(customer.CustomerId, 
                    $"Спасибо за активность! Ваши покупки за месяц: {customer.TotalPurchasesAmount:C}");
            }
        }
        Console.WriteLine("======================================");
    }
}

// ОСНОВНАЯ ПРОГРАММА
Console.WriteLine("================================================");
Console.WriteLine("      СИСТЕМА УПРАВЛЕНИЯ КЛИЕНТАМИ");
Console.WriteLine("================================================");

// 1. СОЗДАНИЕ СЕРВИСОВ
var notificationService = new EmailCustomerNotificationService();
var auditService = new DatabaseAuditService();
var analyticsService = new CustomerAnalyticsService();

// 2. СОЗДАНИЕ СИСТЕМЫ
var customerSystem = new CustomerManagementSystem(notificationService, auditService, analyticsService);

// 3. СОЗДАНИЕ КЛИЕНТОВ С DEPENDENCY INJECTION
Console.WriteLine("\nСОЗДАНИЕ КЛИЕНТОВ:");
Console.WriteLine("========================================");

var vipCustomer = new VipCustomer(1, "Иван VIP Петров", "ivan.vip@example.com",
    notificationService, auditService, monthlyLimit: 75000, personalManager: "Анна Сидорова");

var regularCustomer = new RegularCustomer(2, "Мария Обычная", "maria@example.com",
    notificationService, auditService, creditLimit: 20000, hasOverdraft: true, monthlyBudget: 10000);

var groupCustomer = new GroupCustomer(3, "ИТ-Департамент ООО 'ТехноПрофи'", 
    "it@techprofile.com", "Сергей Козлов", 15, "IT разработка",
    notificationService, auditService);

// 4. ДОБАВЛЕНИЕ КЛИЕНТОВ В СИСТЕМУ
customerSystem.AddCustomer(vipCustomer);
customerSystem.AddCustomer(regularCustomer);
customerSystem.AddCustomer(groupCustomer);

// 5. ДЕМОНСТРАЦИЯ ЯВНОЙ РЕАЛИЗАЦИИ ИНТЕРФЕЙСОВ
Console.WriteLine("\nЯВНАЯ РЕАЛИЗАЦИЯ ИНТЕРФЕЙСОВ:");
Console.WriteLine("========================================");

Console.WriteLine("\nПРОВЕРКА СКИДОК:");
Console.WriteLine($"Скидка VIP клиента: {vipCustomer.GetDiscountPercent()}%");
Console.WriteLine($"Скидка обычного клиента: {regularCustomer.GetDiscountPercent()}%");
Console.WriteLine($"Скидка группового клиента: {groupCustomer.GetDiscountPercent()}%");

Console.WriteLine("\nРАБОТА С БАЛЛАМИ ЛОЯЛЬНОСТИ:");
ILoyaltyEarning vipLoyalty = vipCustomer;
ILoyaltyEarning regularLoyalty = regularCustomer;

vipLoyalty.AddLoyaltyPoints(500, "Бонус за регистрацию");
regularLoyalty.AddLoyaltyPoints(200, "Приветственный бонус");

Console.WriteLine($"Баллы VIP клиента: {vipLoyalty.LoyaltyPoints}");
Console.WriteLine($"Баллы обычного клиента: {regularLoyalty.LoyaltyPoints}");

// 6. ДЕМОНСТРАЦИЯ РАБОТЫ С КЛИЕНТАМИ
Console.WriteLine("\nОПЕРАЦИИ С КЛИЕНТАМИ:");
Console.WriteLine("========================================");

// VIP клиент
vipCustomer.RecordLogin();
vipCustomer.RecordLogin();
vipCustomer.AddPurchase(15000, "Ноутбук премиум");
vipCustomer.AddPurchase(5000, "Смартфон");
vipCustomer.AddSpecialOffer("Бесплатная доставка на год");
vipCustomer.SetPersonalManager("Ольга Смирнова");

// Обычный клиент
regularCustomer.RecordLogin();
regularCustomer.AddPurchase(3000, "Планшет");
regularCustomer.AddToWishlist("Умные часы");
regularCustomer.AddToWishlist("Наушники");
regularCustomer.UpdateCreditLimit(30000);
regularCustomer.SetMonthlyBudget(8000);

// Групповой клиент
groupCustomer.RecordLogin();
groupCustomer.AddMember("Алексей Программист", "Senior Developer");
groupCustomer.AddMember("Мария Дизайнер", "UI/UX Designer");
groupCustomer.AddMember("Сергей Тестировщик", "QA Engineer");
groupCustomer.AddProject("Разработка CRM системы", 500000, true);
groupCustomer.AddProject("Мобильное приложение", 300000, true);
groupCustomer.AllocateBudget(200000);
groupCustomer.SendGroupNotification("Собрание в пятницу в 15:00");

// 7. GENERIC МЕНЕДЖЕР КЛИЕНТОВ
Console.WriteLine("\nGENERIC МЕНЕДЖЕР VIP КЛИЕНТОВ:");
Console.WriteLine("========================================");

var vipManager = new CustomerManager<VipCustomer>(notificationService, auditService, analyticsService);
vipManager.AddCustomer(vipCustomer);
vipManager.ShowAllCustomers();
vipManager.DisplayAnalytics();
vipManager.SendPromotionToAll("Эксклюзивная распродажа для VIP клиентов!");

// 8. СЕРВИС ТРАНЗАКЦИЙ
Console.WriteLine("\nСЕРВИС ТРАНЗАКЦИЙ:");
Console.WriteLine("========================================");

var transactionService = new CustomerTransactionService<Customer>(auditService, notificationService);
transactionService.ProcessPurchase(vipCustomer, 7000, "Монитор 4K");
transactionService.TransferPoints(vipCustomer, regularCustomer, 100);

// 9. ПОКАЗ ПРОФИЛЕЙ КЛИЕНТОВ
Console.WriteLine("\nПРОФИЛИ ВСЕХ КЛИЕНТОВ:");
Console.WriteLine("========================================");

vipCustomer.ViewProfile();
regularCustomer.ViewProfile();
groupCustomer.ViewProfile();

// 10. СТАТИСТИКА СИСТЕМЫ
customerSystem.ShowSystemStatistics();

// 11. МЕСЯЧНЫЙ ОТЧЕТ
customerSystem.RunMonthlyReport();

// 12. ДЕМОНСТРАЦИЯ ПОЛИМОРФИЗМА
Console.WriteLine("\nДЕМОНСТРАЦИЯ ПОЛИМОРФИЗМА:");
Console.WriteLine("========================================");

List<Customer> allCustomers = new List<Customer> { vipCustomer, regularCustomer, groupCustomer };

Console.WriteLine("\nВызов GetFullName() для всех клиентов:");
foreach (var customer in allCustomers)
{
    Console.WriteLine($"{customer.GetType().Name}: {customer.GetFullName()}");
}

Console.WriteLine("\nОбновление email с перегрузкой методов:");
vipCustomer.UpdateEmail("new.ivan@example.com", true);
regularCustomer.UpdateEmail("new.maria@example.com");

Console.WriteLine("\nПросмотр кратких профилей:");
foreach (var customer in allCustomers)
{
    customer.ViewProfile(true);
}

// 13. УПРАВЛЕНИЕ АКТИВНОСТЬЮ КЛИЕНТОВ
Console.WriteLine("\nУПРАВЛЕНИЕ АКТИВНОСТЬЮ КЛИЕНТОВ:");
Console.WriteLine("========================================");

vipCustomer.Deactivate();
vipCustomer.Activate();

// 14. ПРОВЕРКА СКИДОК НА КОНКРЕТНЫЕ ПОКУПКИ
Console.WriteLine("\nПРОВЕРКА СКИДОК НА ПОКУПКИ:");
Console.WriteLine("========================================");

decimal testPurchaseAmount = 1500;
foreach (var customer in allCustomers)
{
    bool canApply = customer.CanApplyDiscount(testPurchaseAmount);
    decimal discountPercent = customer.GetDiscountPercent();
    decimal discountAmount = testPurchaseAmount * discountPercent / 100;
    
    Console.WriteLine($"{customer.GetFullName()}:");
    Console.WriteLine($"  Может применить скидку на {testPurchaseAmount:C}: {(canApply ? "Да" : "Нет")}");
    Console.WriteLine($"  Процент скидки: {discountPercent}%");
    Console.WriteLine($"  Сумма скидки: {discountAmount:C}");
}

Console.WriteLine("\n================================================");
Console.WriteLine("          ПРОГРАММА УСПЕШНО ЗАВЕРШЕНА");
Console.WriteLine("================================================");

      СИСТЕМА УПРАВЛЕНИЯ КЛИЕНТАМИ

СОЗДАНИЕ КЛИЕНТОВ:
Аудит: Клиент 1, Действие: CUSTOMER_CREATED, Детали: Создан клиент: Иван VIP Петров (ivan.vip@example.com), Время: 18:12:09
Приветственное письмо отправлено клиенту ID: 1
Аудит: Клиент 1, Действие: VIP_CUSTOMER_CREATED, Детали: Создан VIP клиент с лимитом ¤75,000.00, Время: 18:12:09
Аудит: Клиент 2, Действие: CUSTOMER_CREATED, Детали: Создан клиент: Мария Обычная (maria@example.com), Время: 18:12:09
Приветственное письмо отправлено клиенту ID: 2
Аудит: Клиент 2, Действие: REGULAR_CUSTOMER_CREATED, Детали: Создан обычный клиент с кредитным лимитом ¤20,000.00, Время: 18:12:09
Аудит: Клиент 3, Действие: CUSTOMER_CREATED, Детали: Создан клиент: ИТ-Департамент ООО 'ТехноПрофи' (it@techprofile.com), Время: 18:12:09
Приветственное письмо отправлено клиенту ID: 3
Аудит: Клиент 3, Действие: GROUP_CUSTOMER_CREATED, Детали: Создан групповой клиент: ИТ-Департамент ООО 'ТехноПрофи' (15 участников), Время: 18:12:09
Аудит: Клиент 1, Действие: CUS